In [10]:
import pandas as pd
import numpy as np
from pycorenlp import StanfordCoreNLP 
import re, time, bisect, math
import TreeBuild as tb

nlp = StanfordCoreNLP('http://localhost:9000')

In [11]:
from random import randint

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)


In [16]:
def _getNLPToks_(rawSentence):
    try:
        output = nlp.annotate(rawSentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse,depparse',
            'outputFormat': 'json'
        })
    except UnicodeDecodeError:
        sentence = unidecode(rawSentence)
        output = nlp.annotate(sentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse,depparse',
            'outputFormat': 'json'
        })
    dependencies = output['sentences'][0]['basicDependencies']
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    return {'deps':dependencies, 'toks':tokens, 'parse':parse}


In [22]:
import pickle

import json, os
with open('stanfordData', 'wb') as fout:
    count = 0
    for row in df_train.iterrows():
        count += 1
        if count > 5:
            break
        
        q1_stanford = _getNLPToks_(row[1]['question1'])
        q2_stanford = _getNLPToks_(row[1]['question2'])

        tree_1 = tb.tree()
        tree_2 = tb.tree()

        # Generate a tree structure
        tb._generateTree_(q1_stanford['parse'], tree_1)
        tb._generateTree_(q2_stanford['parse'], tree_2)

        # Flip the trees
        tb._flipTree_(tree_1)
        tb._flipTree_(tree_2)

        tmp = {'q1': {
                'raw': row[1]['question1'],
                'toks': q1_stanford['toks'],
                'deps': q1_stanford['deps'],
                'parse': tree_1
                },
               'q2': {
                'raw': row[1]['question2'],
                'toks': q2_stanford['toks'],
                'deps': q2_stanford['deps'],
                'parse': tree_2                
                },
               'id':row[1]['id'],
               'is_duplicate':row[1]['is_duplicate']
               }
               
        pickle.dump(tmp, fout, protocol=pickle.HIGHEST_PROTOCOL)
        
        tree_1.clear()
        tree_2.clear()

In [24]:
with open('stanfordData', 'rb') as handle:
    try:
        while True:
            tmp = pickle.load(handle)
            print(tmp['q1']['raw'])
            print(tmp['q2']['raw'])

    except EOFError:
        pass

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?
What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?
Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?
Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?
